You can access the accompanying video here: 

<a href="https://blinks.bloomberg.com/screens/PLYR%20VOD%20360874005"> DataGrid and Barcharts (7 min)</a>

In [3]:
# First we want to import the BQL Library 
import bql
bq = bql.Service()

In [2]:
import pandas as pd
import numpy as np
import random
import bqplot as bqp
import bqwidgets as bqw
from ipywidgets import VBox, HTML, Button

# Set column names to use in dataframe
VALUE_COL_NAME = 'Value'
FILTER_COL_NAME = 'Sector'

# This is where we pasted in our BQL Dataframe

bql_item1 = bq.data.volatility()
bql_item2 = bq.data.gics_sector_name()
bql_items = {VALUE_COL_NAME:bql_item1, FILTER_COL_NAME:bql_item2}

bql_universe = bq.univ.members('SPX Index')

bql_request = bql.Request(bql_universe, bql_items)
bql_response = bq.execute(bql_request)
dataframe = bql.combined_df(bql_response).reset_index()

# Create scales
scale_x = bqp.OrdinalScale()
scale_y = bqp.LinearScale()

# Group the data to plot the median of each sector
median_dataframe = dataframe.groupby(FILTER_COL_NAME).median()

# Create mark
mark_bar = bqp.Bars(x=median_dataframe.index,
                    y=median_dataframe[VALUE_COL_NAME],
                    scales={'x': scale_x, 'y': scale_y},
                    orientation='horizontal',
                    padding=0.15,
                    colors=['#1B84ED'],
                    interactions={'click': 'select'},
                    unselected_style={'opacity': .5},
                    tooltip=bqp.Tooltip(fields=['y'], show_labels=False))

# Start plot with all bars selected
mark_bar.selected = list(range(len(mark_bar.x)))

# Create Axes
axis_x = bqp.Axis(scale=scale_x, orientation='vertical')
axis_y = bqp.Axis(scale=scale_y, label='Median of values per category')

# Create Figure
figure = bqp.Figure(marks=[mark_bar],
                    axes=[axis_x, axis_y],
                    title='Horizontal bar plot w/ linked data grid',
                    title_style={'font-size': '22px'},
                    fig_margin={'top': 60, 'bottom': 50,
                                'left': 90, 'right': 40},
                    layout={'width': '100%', 'height': '300px'})

# Create grid and "reset" button
datagrid = bqw.DataGrid(data=dataframe, layout={'height': '200px',
                                                'width': '99%',
                                                'overflow_x': 'hidden'})
reset_button = Button(description='Show all values')

# Callback to handle selections on bar chart
def on_bar_click(evt=None):
    if evt is not None and evt['new'] is not None:
        selected = [mark_bar.x[i] for i in evt['new']]
        filter_df = dataframe[dataframe[FILTER_COL_NAME].isin(selected)]
        datagrid.data = filter_df

# Callback to handle button clicks
def on_button_click(evt=None):
    datagrid.data = dataframe
    mark_bar.selected = list(range(len(mark_bar.x)))

# Attach listeners
mark_bar.observe(on_bar_click, names='selected')
reset_button.on_click(on_button_click)

visualization = VBox([figure, reset_button, datagrid],
                     layout={'overflow_x': 'hidden'})

# Display the figure
visualization
